# Programmentwurf DataScience

## 1. Business Understanding
**Aufgabenstellung**: _Formulieren Sie ein Ziel oder mehrere Ziele nach dem CRISP-DM Prozess, die für __einen Immobilienverein von Privatpersonen__ sinnvoll sind. Diese Personen kaufen und verkaufen Häuser zur Eigennutzung oder für die Familie, manchmal mehrfach. Beginnen Sie mit der Idee „Wir brauchen mehr Verständnis und eine Vorhersage des Verkaufspreises (`Z_Verkaufspreis`)!“, welche auf jeden Fall zu bearbeiten ist. Geben Sie dies in Ihrem Jupyter-Notebook als Markup an (max. ½ Seite Text)._

### Ziele des Business Understandings
- Was sind die Ziele auf Geschäftsebene?
- Welche Anforderungen an das Ergebnis gibt es?
- Welche offenen Fragen sollen beantwortet werden?
- Wie könnten beispielhafte Antworten aussehen?

Das grundsätzliche Ziel ist die Evaluation der Rechtfertigkeit eines Hauspreises. Es soll vermieden werden, Häuser zu teuer anzukaufen, beziehungsweise zu billig zu verkaufen.
Das Ergebnis der Datenanalyse soll eine verlässlige Vorhersage eines Hauspreises aufgrund gegebener Attribute liefern. Dadurch ergeben sich die Fragen:
1. Durch welche Attribute wird der Preis eines Hauses festgemacht?
2. Wie stark beeinflusst ein Attribut den Preis?
3. Gibt es eine Möglichkeit den Wert eines Hauses rentabel und vergleichsweise einfach zu steigern? Sprich welche Investitionen beeinflussen den Hauspreis positiv und sind gleichzeitig vergleichsweise billig. Diese Möglichkeiten sollen auch bei einem Kaufvorschlag in Betracht gezogen werden.

Außerdem soll das Verfahren eine einfache Möglichkeit liefern, einen Hauspreis aufgrund von gegebenen Informationen schätzen zu können. Diese Schätzung soll auch ohne Kenntnisse in DataScience möglich sein, beispielsweise durch eine Formel. Diese Formel wird durch die Vorhersage des Programms bestimmt und sollte möglichst universell einsetzbar sein. Ein Beispiel für eine Formel wäre `Grundfläche * 1000 + AnzahlZimmer * 10 000 + ...`. Die Formel besteht aus den wichtigsten Attributen (Fragestellungen 1. und 2. von oben) und einem statischen Faktor.

Zusätzlich soll der Datenbestand als Grundlage für eine Anwendung dienen, in welche die Eigenschaften und der Preis des Hauses eingetragen werden können. Diese Anwendung führt eine Kategorisierung in "Direkt kaufen", "Abwarten", "Anschauen" und "Schlechtes Angebot" durch. Dadurch soll der Aufwand für die Privatpersonen reduziert werden.

Neben der Vorhersage und Abhängigkeit des Preises von Attributen sollen auch Zusammenhänge der Attribute untereinander gefunden werden.

## 2. Data Exploration und Analyse
**Aufgabenstellung**: _Laden und untersuchen Sie den Datensatz in data_for_training.csv nach den Regeln wie in der Vorlesung gelehrt. Ändern Sie hierbei nicht die einzulesende csv-Datei! Schreiben Sie die wichtigsten Erkenntnisse für die in Aufgabe 1 definierten Ziele als Summary auf (max. ½ Seite Text). Passen Sie ggfs. Ihre Ziele im Business Understanding an. Kommentieren Sie Rückfragen für Fachexperten sowie mögliche zusätzliche Datenquellen und Auswertungen, die Sie damit ausführen würden._

### Laden der Test und Trainingsdatensätze

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
test_data = pd.read_csv('data_for_testing_test.csv', delimiter=";")
train_data = pd.read_csv('data_for_training.csv',delimiter=";").drop(columns="A_Index")

FileNotFoundError: FileNotFoundError: [Errno 2] No such file or directory: 'data_for_testing_test.csv'

**! Attribute vom Typ "object" sind in Stringrepresentation und müssen codiert/quantifiziert/formatiert werden !**

In [0]:
object_columns = train_data.dtypes[train_data.dtypes == 'object'].index
object_columns

In [0]:
train_data

In [0]:
for column in object_columns:
    print(column,":", train_data.loc[:,column].unique(), "\n")

* **Ausbaustufe**: Transformation zu Float oder Label Encoding --> Reihenfolge beibehalten
* **Bebauungsdichte**: Label Encoding
* **Besonderheiten**: Sehr dünn besetzt --> One Hot Encoding ? 0/1 ?
* **Dachtyp**: Label
* **Fassadenqual**: Label Encoding,  Einteilung : Annahme: sehr gut gleich weit von sehr schlecht, durschnitt bei 0, z.B. [Schlecht, Durch, Gut, Sehr gut] = [-2, 0, 1, 2]
* **Fassadenzustand**: Wie oben, aber direkte Abbildung ['Sehr Schlecht' 'Schlecht' 'Durchschnitt' 'Gut' 'Sehr gut' ] = [-2, -1, 0, 1, 2]
* **Kellerhoehe**: Label Encoding: ['Sehr Schlecht' 'Schlecht' 'Durchschnitt' 'Gut' 'Sehr gut' ] = [-2, -1, 0, 1, 2] ---> NAN ?
* **Lage**: Label Encoding ?

In [0]:
train_data.corr()

Verteilung der Hauspreise

In [0]:
_ = sns.pairplot(train_data.loc[:,["Baujahr", "EG_qm", "Grundstueck_qm", "Keller_qm", "Umgebaut", "Wohnflaeche_qm"]])

Diese Scatterplotmatrix verdeutlicht vor allem die Zusammenhänge von Attributen zueinander.

Die Histogramme auf der Diagonale zeigen:
- Ein Großteil der Häuser wurde in den letzten zwei Jahrzehnten erbaut.
- Die Flächen (_EG_qm_, _Grundstueck_qm_, _Keller_qm_ und _Wohnflaeche_qm_) sind normalverteilt, aber leicht verschoben.
- 1960 wurden überraschend viele Häuser umgebaut.

Die Deckelung (Dachform) zwischen _Umgebaut_ und _Baujahr_ stellt all die Häuser dar, die noch nicht umgebaut wurden. Auch lässt sich erkennen, dass ein Großteil der Häuser nach der Jahrtausendwende umgebaut wurden.

In _Keller_qm_ und _EG_qm_ lässt sich ein aufsteigender Trend über die Jahre beobachten (_Baujahr_). Dieser Trend ist bei _Wohnflaeche_qm_ allerdings nicht sichtbar.

Die Winkelhalbierende zwischen _EG_qm_ und _Wohnflaeche_qm_ repräsentiert diejenigen Häuser, die nur aus einem Erdgeschoss bestehen.

Jedes Haus mit mehr als 200 m² Wohnfläche besitzt einen Keller.

Die beiden Ausreißer in _Grundstueck_qm_ liegen in fast allen anderen Attributen ebenfalls eng beeinander mit der Ausnahme, dass eines der beiden Häuser renoviert wurde. Gleiches gilt für _Keller_qm_ und _EG_qm_. Auch hier sind die Ausreißer oft beeinander.

### Histogramm über den Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(15, 10))
axes.set_title("Histogramm über den Verkaufspreis")
axes.set_xlabel("Verkaufspreis in Euro")
axes.set_ylabel("Anzahl der Häuser")
_ = train_data["Z_Verkaufspreis"].hist(bins=100, ax=axes)

Das Histogramm über den Verkaufspreis zeigt, dass ein Großteil der Häuser in der Preisspanne zwischen 100 000€ und 300 000€ verkauft wird.

### Korrelationsmatrix über alle Dimensionen des Datensatzes

In [0]:
plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(train_data.corr(), dtype=bool))
heatmap = sns.heatmap(train_data.corr(), mask=mask, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

Die Korrelationsmatrix zeigt, dass der Verkaufspreis (_Z_Verkaufspreis_) von vielen Attributen abhängig ist.

Neben der Wohnfläche (_Wohnfläche_qm_, bestehend aus _EQ_qm_ und _Keller_qm_) korreliert die Anzahl der Garagen überraschend hoch mit dem Verkaufspreis. Außerdem korrelieren die Anzahl der Bäder (_Baeder_), das _Baujahr_ (zusammen mit _Umgebaut_) und die Anzahl der Kamine (_Kamine_) positiv mit dem Verkaufspreis.
Entgegen der Erwartungen spielt die Anzahl der Zimmer eine eher kleine Rolle in der Bestimmung des Verkaufspreises. Auch die Gesamtfläche des Grundstückes spielt eine eher zu vernachlässigende Rolle mit einer Korrelation von 0,26.

Die größte Korrelation tritt zwischen _Keller_qm_ und _EG_qm_ auf, was sich durch den baulich bedingten Zusammenhang zwischen Keller und Erdgeschoss erklären lässt.

Die Korrelation von Verkaufsjahr und Verkaufspreis kann durch die steigenden Immobilienpreise und die zusätzliche Inflation erklärt werden.

In [0]:
train_data_without_objects = train_data.select_dtypes(exclude=['object'])
prices = train_data_without_objects["Z_Verkaufspreis"]

### Violinplot zur Analyse des fehlenden Zusammenhangs zwischen Verkaufspreis und der Anzahl an Zimmern

In [0]:
selection = train_data_without_objects.loc[:,["Z_Verkaufspreis", "AnzahlZimmer"]]

plt.figure(figsize=(20, 10))
axes = sns.violinplot(x="AnzahlZimmer", y="Z_Verkaufspreis", data=train_data, color="blue")
axes.set_title("Violinplot über den Verkaufspreis abhängig von der Anzahl an Zimmern")
axes.set_xlabel("Anzahl der Zimmer")
_ = axes.set_ylabel("Verkaufspreis in Euro")

Wie bereits in der Korrelationsmatrix erkannt, ist der Zusammenhang zwischen Verkaufspreis und der Anzahl an Zimmern sehr gering.

Normalerweise würde man hier eine stetige Steigerung des Verkaufspreises erwarten, je mehr Zimmer in dem Haus vorhanden sind. Dem ist aber nicht so. Häuser mit zwei Zimmern kosten im Durchschnitt ähnlich viel wie Häuser mit sechs Zimmer.
Da nur ein Haus mit acht Zimmern vorhanden ist, lässt sich dieser Wert vernachlässigen.

### Scatterplots zur Darstellung des Zusammenhangs einzelner Attribute mit dem Verkaufspreis

In [0]:
numeric_columns = ["Baujahr", "EG_qm", "Grundstueck_qm", "Keller_qm", "Umgebaut", "Wohnflaeche_qm"]
numerical_data = train_data_without_objects.loc[:,numeric_columns]

def show_scatter(columnName, drop=None):
    plt.figure(figsize=(10,5))
    if drop is None:
        column_compare = train_data_without_objects[columnName]
        plt.scatter(column_compare, prices)
    else:
        column_compare = train_data_without_objects[columnName].drop(drop)
        plt.scatter(column_compare, prices.drop(drop))

    plt.title(f"Verkaufspreis vs. {columnName}")
    plt.xlabel(columnName)
    plt.ylabel("Verkaufspreis in Euro")
    plt.show()

In [0]:
show_scatter("Baujahr")

Hier lässt sich gut erkennen, dass neuere Häuser tendenziell einen höheren Verkaufspreis haben als ältere.

In [0]:
show_scatter("Umgebaut")

Dieser Scatterplot ähnelt dem obigen über das Baujahr stark und verdeutlicht den Zusammenhang zwischen Verkaufspreis und dem Jahr noch deutlicher. Häuser, die im obigen Graphen herausstechen (z. B. 450 000 € bei Baujahr 1900) sind im _Umgebaut_-Plot nicht mehr vorhanden. Daraus lässt sich schließen, dass der Umbau (/Renovierung) den Wert eines Hauses steigert. Außerdem wird die Aussage "Neuere Häuser sind teurer" auch für alte, aber renovierte, Häuser wahr.

In [0]:
show_scatter("EG_qm")

Die starke Korrelation zwischen _EG_qm_ und _Z_Verkaufspreis_ aus der obigen Korrelationmatrix wird durch diesen Scatterplot verdeutlicht. Entfernt man die Ausreißer bei EG_qm > 400, ergibt sich der folgende Plot:

In [0]:
show_scatter("EG_qm", [737, 216])

In [0]:
show_scatter("Keller_qm", [216, 737])

**Hier wurden zwei Ausreißer bereits entfernt. Bei den Ausreißern handelt es sich um die selben wie bei _EG_qm_**

Dieser Plot ist nahezu deckungsgleich mit dem Plot über _EG_qm_. Da diese beiden Attribute eine Korrelation von 0,81 haben, ist das keine Überraschung. Allerdings stechen die Einträge bei 0 m² Kellerfläche ins Auge.

In [0]:
show_scatter("Wohnflaeche_qm")

Da sich die Wohnfläche unter anderem aus _EG_qm_ und _Keller_qm_ zusammensetzt, ist hier ein ähnlicher Zusammenhang zum Verkaufspreis zu erkennen. Je mehr Wohnfläche ein Haus bietet, desto teurer kann es verkauft werden. Die drei Ausreißer in der rechten unteren Ecke sind den Ausreißern in _EG_qm_ und _Keller_qm_ geschuldet. Entfernt man die beiden Ausreißer und einen neuen, sieht der Plot wie folgt aus:

In [0]:
show_scatter("Wohnflaeche_qm", [737, 216] + [543])

In [0]:
show_scatter("Grundstueck_qm")

Auch hier stechen die Ausreißer bei 16 000 m² Grundstück ins Auge. Allerdings gibt es viele große Grundstücke mit einem vergleichsweise niedrigen Verkaufspreis. Ohne Ausreißer wirkt der Zusammenhang zwischen Preis und Grundfläche stärker, allerdings gibt es noch sehr viele große Grundstücke mit einem vergleichsweise niedrigen Preis:

In [0]:
show_scatter("Grundstueck_qm", [757, 536])

In [0]:
columns_for_boxplot =  list(set(train_data_without_objects.columns) - {"Baujahr", "EG_qm", "Grundstueck_qm", "Keller_qm", "Umgebaut","Wohnflaeche_qm","Z_Verkaufspreis"})
columns_for_boxplot

### Kellerqualität vs. Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss der Kellerqualität auf den Verkaufspreis")
ax = sns.boxplot(x="KellerQual", y="Z_Verkaufspreis", data=train_data, color="blue")
ax.set(xlabel='Keller Qualität (1:Sehr Schlecht, 2:Schlecht, 3:Durchschnitt, 4:Gut, 5:Sehr Gut)', ylabel='Verkaufspreis in Euro')
plt.show()

Es ist kein wesentlicher Zusammenhang der Kellerqualität und des Verkaufpreises ersichtlich.

### Verkaufsjahr vs. Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss des Verkaufsjahres auf den Verkaufspreis")
ax = sns.boxplot(x="Verkaufsjahr", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Verkaufsjahr', ylabel='Verkaufspreis in Euro')
plt.show()

Hier ist ein leichter Aufwärtstrend zu erkennen, Häuser die später verkauft wurden, wurden in der Regel etwas teurer verkauft. Dies hängt sehr wahrscheinlich mit allgemein steigenden Hauspreisen und/oder der Inflation zusammen --> Rückfrage.

### Anzahl der Bäder im UG vs. Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss des Anzahl von Bädern im UG auf den Verkaufspreis")
ax = sns.boxplot(x="BaederUG", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Anzahl der Bäder im UG', ylabel='Verkaufspreis in Euro')
plt.show()

### Anzahl Kamine vs. Hauspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss der Anzahl an Kaminen auf den Verkaufspreis")
ax = sns.boxplot(x="Kamine", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Anzahl der Kamine', ylabel='Verkaufspreis in Euro')
plt.show()

Bei der Anzahl an Kaminen ist ein sehr starken Zusammenhang mit dem Verkaufpreis zu erkennen, da ein Haus mit mehr Kaminen in der Regel auch ein größeres Haus ist. Dies ist im folgeden Plot zu erkennen. Bei der Kategorie 4 Kamine handelt es sich nur um einen Datensatz.

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Zusammenhang der Anzahl an Kaminen und der Größe des Erdgeschosses")
ax = sns.violinplot(x="Kamine", y="EG_qm", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Anzahl der Kamine', ylabel='Größe des Erdgeschosses in qm')
plt.show()

### Einfluss der Anzahl an Bädern auf den Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss der Anzahl an Bädern auf den Verkaufspreis")
ax = sns.boxplot(x="Baeder", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Anzahl der Bäder', ylabel='Verkaufspreis in Euro')
plt.show()

Hier ist zu beachten, dass null Bäder nur 10 Datensätze haben, wobei es sich um Fehler handeln könnte, da Häuser typischerweise ein Bad haben. Ebenfalls haben nur vier Datensätze vier Bäder, was auch nicht sehr aussagekräftig ist. Bei den Anzahlen eins bis drei Bädern ist jedoch ein starker Anstieg des Verkaufspreises mit der Anzahl an Bädern zu erkennen.

### Einfluss der Anzahl an Garagen auf den Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss der Anzahl an Garagen auf den Verkaufspreis")
ax = sns.violinplot(x="Garagen", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Anzahl der Garagen', ylabel='Verkaufspreis in Euro')
plt.show()

Hier ist zu erkennen, dass ein Haus mit einer Garage nur eine geringe Wertsteigerung im Vergleich zu einem Haus mit keiner Garage erfährt. Jedoch ist ein Haus mit zwei Garagen im Median ca. 50k mehr Wert als ein Haus mit einer Garage. Ein Haus mit drei Garagen ist im Median sogar 100k mehr Wert als ein Haus mit zwei Garagen. Für vier Garagen gibt es nur 13 Datensätze, was wenig aussagekräftig ist.

### Einfluss des Gesamteindrucks auf den Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss der Gesamteindrucks auf den Verkaufspreis")
ax = sns.boxplot(x="Gesamteindruck", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Gesamteindruck (1:geringster, 5:bester)', ylabel='Verkaufspreis in Euro')
plt.show()

Hier ist zu beobachten, dass der Preis im Median umso höher ist, je höher der Gesamteindruck ist. Gleichzeitig ist aber auch die Preisspanne noch oben deutlich höher (Oberes Quartil und Whisker), bei einem sehr guten Gesamteindruck. Dies ist vor allem herrausstechend für den Gesamteindruck 4. Jedoch erhöht sich die untere Preisspanne (unteres Quartil) nicht so stark mit steigendem Gesamteindruck. Es könnte der Fall sein, dass der Gesamteindruck den möglichen Verhandlungspreis in die Höhe treibt, dieser wird aber nicht immer voll ausgereizt.

Ausreiser/Extreme:
    * Bei Gesamteindruck 0: Gibt es nur drei Messwerte, der Wert null ist jedoch nicht in der Skala definiert
    * Bei Gesamteindruck 5: n = 2, sehr geringe Sample size

### Einfluss des Verkaufsmonat auf den Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss des Verkaufsmonats auf den Verkaufspreis")
ax = sns.boxplot(x="Verkaufsmonat", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Verkaufsmonat', ylabel='Verkaufspreis in Euro')
plt.show()

Kein signifikanter Zusammenhang zwischen Verkaufsmonat und Verkaufspreis, jedoch wird ersichtlich, dass im Median Häuser am teuersten im Januar verkauft werden. Außerdem gibt es in den Sommermonaten Mai bis Juli die meisten Ausreiser, wovon man ausgehen kann, dass in diesen Monaten eher sehr hochpreisige Häuser verkauft werden.

### Zusammenhang der Zimmeranzahl mit dem Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss der ANzahl der Zimmer auf den Verkaufspreis")
ax = sns.boxplot(x="AnzahlZimmer", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Anzahl der Zimmer', ylabel='Verkaufspreis in Euro')
plt.show()

Bei ursrünglicher Betrachtung scheint die Anzahl der Zimmer keinen wesentlichen linearen Zusammenhang mit dem Verkaufspreis zu haben.
Dies hängt damit zusammen, dass AnzahlZimmer=0,5,6,8 weniger representativ sind, da sie nur n=6,32,16,1 Datensätze enthalten. Für die Datensätze mit einer representativen Anzahl n=2,3,4 gibt es einen geringen Anstieg des Median und des unteren Quartils.

Ausreiser/Extreme:
* 0 Zimmer (Wie ist das möglich ?): n=6
* 8 Zimmer: n=1

### Zusammenhang Toiletten mit dem Verkaufspreis

Neues Feature: Toiletten = ToilettenEG + ToilettenUG

In [0]:
# Korrelation von Verkaufspreis mit ToilettenEg/ ToilettenUG
eg_corr = train_data["ToilettenEG"].corr(train_data["Z_Verkaufspreis"])
ug_corr = train_data["ToilettenUG"].corr(train_data["Z_Verkaufspreis"])
print(f"Correlation with Z_Verkaufspreis:\n  - ToilettenEG:  {np.round(eg_corr,3)}\n  - ToilettenUG: {np.round(ug_corr,3)}")

In [0]:
train_data["Toiletten"] = train_data["ToilettenEG"] + train_data["ToilettenUG"]

In [0]:
# New Correlation
print(f"New Corr: -Toiletten, VErkaufspreis: {train_data['Toiletten'].corr(train_data['Z_Verkaufspreis'])}")

Kombination der Features ToilettenEG und ToilettenUG bringt keine große Verbesserung für die Korrelation mit dem Verkaufspreis

In [0]:
_, axes = plt.subplots(figsize=(20, 10))
axes.set_title("Einfluss der Anzahl an Toiletten im EG auf den Verkaufspreis")
ax = sns.boxplot(x="ToilettenEG", y="Z_Verkaufspreis", data=train_data, ax=axes, color="blue")
ax.set(xlabel='Anzahl Toiletten im EG', ylabel='Verkaufspreis in Euro')
plt.show()

Anzahl Datensätze mit zwei Toiletten im EG: 17
Es ist ein leichter Ansteig des Verkaufspreises anhand der Anzahl an Toiletten im EG zu erkennen.

Ausreiser:
* 0 --> nur 3 Datensätze
* 5 --> nur 2 Datensätze

Insgesamt deutlich höhere Preise (Nach Median) + Preisspanne nach oben für einen höhere Gesamteidruck

### Korrelation der einzelnen Attribute

TODO: Hier noch korrelationen unabhängig von Preis darstellen.

Preisspanne nach oben deutlich höher

### Zusammenfassung der Erkenntnisse

#### Rückfragen für Fachexperte (z.B. Immobilienmakler):

* Warum werden im Sommer mehr Häuser verkauft
* Wie lange dauert es durchschnittlich, bis Haus renoviert wird ?
* Wie ist das Durchschnittliche Einkommen in den Stadtteilen ?

## 3. Data Preparation

**Aufgabenstellung**: _Bereinigen Sie die Daten und führen Sie ein sinnvolles Feature Engineering durch. Hinweis: Das kann auch für Punkt 2 bereits relevant sein (führen Sie das dann hier dann nochmals zusammenfassend auf)._

Die folgenden Spalten beötigen definitiv bearbeitung, sie sollten Numerisch gesetzt/ Codiert werden

In [0]:
object_columns

#### Ausbaustufe: Label Encoding

In [0]:
train_data["Ausbaustufe"].unique()

In [0]:
# Ausbaustufe
Ausbaustufe_mapping = {"Sonstige":0,"1 Ebene":1.0,"1,5 Ebenen":1.5,"2 Ebenen":2.0,"2,5 Ebenen":2.5}
train_data["Ausbaustufe"] = train_data["Ausbaustufe"].replace(Ausbaustufe_mapping)

#### Bebauungsdichte

In [0]:
train_data["Bebauungsdichte"].unique()

Besonders interessant ist hier die Seelage, diese gehört nicht in die Kategorie "Bebauungsdichte", jedoch ist es sehr wahrscheinlich, dass ein Haus in Seelage mehr kostet, als ein Haus ohne Seelage.
Die Seelage wird extrahiert, und es wird ein neues Feature für sie angelegt.

In [0]:
# neues Feature für Seelage: 1 wenn Seelage,0 sonst
Seelage = pd.Series(np.where(train_data["Bebauungsdichte"] == "Seelage", 1.0, 0.0))
train_data["Seelage"] = Seelage

In [0]:
# ersetzte Seelage als unbekannt, Label Encoding
Bebauungsdichte_mapping = {"Seelage":0.0,"Unbekannt":0.0,"Niedrig":1.0,"Mittel":2.0,"Hoch":3.0}
train_data["Bebauungsdichte"] = train_data["Bebauungsdichte"].replace(Bebauungsdichte_mapping)

#### Besonderheiten

In [0]:
train_data["Besonderheiten"].unique()

Hier können ebenfalls mittels Hot One Encoding zwei Features generiert werden, so sind NaN werte auch nicht wichtig

In [0]:
# neues Feature für Neuverkauf: 1 wenn Neuverkauf,0 sonst
Neuverkauf = pd.Series(np.where(train_data["Besonderheiten"] == "Neuverkauf", 1.0, 0.0))
train_data["Neuverkauf"] = Neuverkauf

# neues Feature für Wasseranschluss: 1 wenn Wasseranschluss, 0 sonst
Wasseranschluss = pd.Series(np.where(train_data["Besonderheiten"] == "kein Wasseranschluss", 0.0, 1.0))
train_data["Wasseranschluss"] = Wasseranschluss

# Besonderheiten Spalte löschen
train_data = train_data.drop(columns=["Besonderheiten"])

#### Dachtyp
klassisches labelencoding

In [0]:
train_data["Dachtyp"].unique()

In [0]:
from sklearn import preprocessing

In [0]:
dachtyp_encoder = preprocessing.LabelEncoder()
_= dachtyp_encoder.fit(train_data["Dachtyp"])
train_data["Dachtyp"] = dachtyp_encoder.transform(train_data["Dachtyp"])

**Fassadenqual**: Label Encoding,  Einteilung : Annahme: sehr gut gleich weit von sehr schlecht, durschnitt bei 0, z.B. [Schlecht, Durch, Gut, Sehr gut] = [-2, 0, 1, 2]
* **Fassadenzustand**: Wie oben, aber direkte Abbildung ['Sehr Schlecht' 'Schlecht' 'Durchschnitt' 'Gut' 'Sehr gut' ] = [-2, -1, 0, 1, 2]
* **Kellerhoehe**: Label Encoding: ['Sehr Schlecht' 'Schlecht' 'Durchschnitt' 'Gut' 'Sehr gut' ] = [-2, -1, 0, 1, 2] ---> NAN ?
* **Lage**: Label Encoding ?

#### Fassadenqual
--> LAbelencoding

In [0]:
train_data["Fassadenqual"].unique()

In [0]:
sehr_schlect_sehr_gut_mapping = {"Sehr Schlecht":-2.0,"Schlecht":-1.0,"Durchschnitt":0.0,"Gut":1.0,"Sehr gut":2.0}
train_data["Fassadenqual"] = train_data["Fassadenqual"].replace(sehr_schlect_sehr_gut_mapping)

#### Fasadenzustand
--> Labelencoding

In [0]:
train_data["Fassadenzustand"].unique()

In [0]:
train_data["Fassadenzustand"] = train_data["Fassadenzustand"].replace(sehr_schlect_sehr_gut_mapping)

#### Kellerhoehe
--> Labelencoding

In [0]:
train_data["Kellerhoehe"].unique()

In [0]:
train_data["Kellerhoehe"] = train_data["Kellerhoehe"].replace(sehr_schlect_sehr_gut_mapping)

#### Lage
--> Label encoding

In [0]:
# auffüllen von Nullwerten mit "unbekannt"
train_data["Lage"] = train_data["Lage"].fillna(value="Unbekannt")

Lage_encoder = preprocessing.LabelEncoder()
_= Lage_encoder.fit(train_data["Lage"])
train_data["Lage"] = Lage_encoder.transform(train_data["Lage"])

#### Typ

In [0]:
# auffüllen von Nullwerten mit "unbekannt"
train_data["Typ"] = train_data["Typ"].fillna(value="Unbekannt")

Typ_encoder = preprocessing.LabelEncoder()
_= Typ_encoder.fit(train_data["Typ"])
train_data["Typ"] = Typ_encoder.transform(train_data["Typ"])

In [0]:
plt.figure(figsize=(22, 15))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(train_data.corr(), dtype=bool))
heatmap = sns.heatmap(train_data.corr(), mask=mask, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

## 4. Modeling - Regression mit Interferenz

**Aufgabenstellung**: _Führen Sie mit geeigneten Verfahren der Regression (Linear / Lasso / Ridge) eine Vorhersage des Preises (Z_Verkaufspreis) durch, der akzeptabel in der Evaluation abschneidet und Verständnis ermöglicht. Erklären Sie die identifizierten Zusammenhänge menschenverständlich als Text (mit nachvollziehbarer Anzahl der Merkmale)._

## 5. Modeling - Best of Class

**Aufgabenstellung**: _Vergleichen und optimieren Sie ggfs. ein oder mehrere andere Verfahren zur Vorhersage des Verkaufspreises (Z_Verkaufspreis). Gehen Sie vor wie in der Vorlesung gelehrt mit Trainings- und Validierungsdaten. Vergleichen und optimieren Sie. Interpretieren Sie das Ergebnis und den Einfluss der Dimensionen (falls möglich)._

## 6.Evaluation und Test

**Aufgabenstellung**: _Schließen Sie die Aufgabe mit einer finalen Evaluation der Vorhersagequalität ab. Stellen Sie sicher, dass die Testdatei (data_for_testing_test.csv) ladbar ist und mit zurückgehaltenen Testdaten data_for_test.csv ersetzt werden kann, um einen Benchmark zu erstellen. Kommentieren Sie die Stelle mit dem Kommentar „#HIER DATEINAMEN ERSETZEN“. Geben Sie für die Testdaten aus: R2, MSE, RMSE, MAPE, MAX._

## 7. Deployment

**Aufgabenstellung**: _Erstellen Sie eine Anleitung oder Handreichung für den Immobilienverein aus Aufgabe 1 basierend auf allen Erkenntnissen. Sie können Markup und erklärende Visualisierungen innerhalb des .ipynb nutzen. Dies soll alle
für den Verein wichtigen Erkenntnisse zusammenfassen (auch wenn dadurch Redundanz in der Abgabe entsteht) und maximal 2 Seiten im pdf-Ausdruck umfassen, welche komplett eigenständig lesbar sein sollen._

## Add-On: Modeling - Klassifikation

**Aufgabenstellung**: _Versuchen Sie eine Vorhersage der Kamine (ja / nein, alternativ: Anzahl der Kamine, Feld: Kamine). Evaluieren Sie die Vorhersage. Dieser Punkt fließt nicht in alle anderen Aufgabenteile ein._

## Bewertungskriterien

1. Fachliche Bewertung (50%): Korrektheit, Lösungsqualität und Eleganz sowie Klarheit und Umfang der Betrachtung, Umsetzung von Data Science wie in der Vorlesung gelehrt in einem Code-Prototyp, korrekte Verwendung von wichtigen Funktionen / Bibliotheken, Güte der Endlösung, Nutzung der erworbenen Kenntnisse aus der Vorlesung, Vollständigkeit der Lösung in Bezug auf die Aufgabenstellung
2. Dokumentation (50%): Dokumentation des Vorgehens der Datenauswertung im Sinne von Data Science, Codekommentare wie in der Informatik üblich wo notwendig, Qualität der Diagramme, Markup, Texte, pdf